In [1]:
import numpy as np
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os
tf.keras.backend.set_floatx('float64')

In [2]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=6000)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
data = np.load('/home/ML4NO/Data/n1000000_0910_all_flat.npz')

In [4]:
data_all = np.column_stack([data['ve_dune'], data['vu_dune'], data['vebar_dune'], data['vubar_dune'], data['ve_t2hk'], data['vu_t2hk'], data['vebar_t2hk'], data['vubar_t2hk']])
target = data['theta23']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [5]:
clf = ak.StructuredDataRegressor(overwrite=True, max_trials=50)
clf.fit(x_train, y_train,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
       )

Trial 50 Complete [00h 01m 01s]
val_loss: 0.007453565951436758

Best val_loss So Far: 0.005114410072565079
Total elapsed time: 00h 48m 10s
INFO:tensorflow:Oracle triggered exit
Epoch 1/20
157/157 [==============================] - 5s 16ms/step - loss: 863.1394 - mean_squared_error: 863.1394
Epoch 2/20
157/157 [==============================] - 3s 16ms/step - loss: 16.2109 - mean_squared_error: 16.2109
Epoch 3/20
157/157 [==============================] - 3s 17ms/step - loss: 7.8313 - mean_squared_error: 7.8313
Epoch 4/20
157/157 [==============================] - 3s 16ms/step - loss: 7.6371 - mean_squared_error: 7.6371
Epoch 5/20
157/157 [==============================] - 3s 16ms/step - loss: 4.7101 - mean_squared_error: 4.7101
Epoch 6/20
157/157 [==============================] - 3s 16ms/step - loss: 6.4930 - mean_squared_error: 6.4930
Epoch 7/20
157/157 [==============================] - 3s 16ms/step - loss: 5.2365 - mean_squared_error: 5.2365
Epoch 8/20
157/157 [====================

In [6]:
model = clf.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 304)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 304)               0         
_________________________________________________________________
normalization (Normalization (None, 304)               609       
_________________________________________________________________
dense (Dense)                (None, 32)                9760      
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               8448      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 256)               0     

In [7]:
model.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=20,
           verbose=1,
           shuffle = True
         )

Epoch 1/20
12516/12516 [==============================] - 223s 18ms/step - loss: 0.1808 - mean_squared_error: 0.1808 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 2/20
12516/12516 [==============================] - 219s 18ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 3/20
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 4/20
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 5/20
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0041 - val_mean_squared_error: 0.0041
Epoch 6/20
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0041 - mean_squared_error: 0.0041 - val_loss: 0.0041 - val_mean_squared_

In [8]:
index = 1
while os.path.isfile('/home/ML4NO/ML/Regression/models/0910_theta23_{}.h5'.format(index)):
    index += 1
model.save('/home/ML4NO/ML/Regression/models/0910_theta23_{}.h5'.format(index))

In [9]:
scale_steps = np.logspace(-3, 0, 30)
before_train_loss = []
after_train_loss = []

for scale in scale_steps:
    x_train2_gen = np.random.normal(x_train2, np.sqrt(x_train2)*scale)
    x_test_gen = np.random.normal(x_test, np.sqrt(x_test)*scale)

    before_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

    model.fit(x_train2_gen, y_train2,
               validation_split = 0.1,
               batch_size=64,
               epochs=5,
               verbose=1,
               shuffle = True
             )

    after_train_loss.append(model.evaluate(x_test_gen, y_test)[0])

3125/3125 [==============================] - 38s 12ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 1/5
12516/12516 [==============================] - 219s 18ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 2/5
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 3/5
12516/12516 [==============================] - 218s 17ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 4/5
12516/12516 [==============================] - 219s 18ms/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0050 - val_mean_squared_error: 0.0050
Epoch 5/5
3125/3125 [==============================] - 37s 12ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 1/5
12516/12516 [==============================] - 220s 18ms/step - loss: 0.0056 - mean_squared_error: 0.0

In [10]:
model_index = index
index = 1
path = '/home/ML4NO/ML/Regression/models_furthurTrain/0910_theta23_{}_{}.h5'
while os.path.isfile(path.format(model_index, index)):
    index += 1
model.save(path.format(model_index, index))
outfile = {'scale_steps': scale_steps,
           'before_train_loss': before_train_loss,
           'after_train_loss' :after_train_loss}
np.save(file = '/home/ML4NO/ML/Regression/models_furthurTrain/0910_theta23_{}_{}_result.npy'.format(model_index, index),
        arr = outfile)

In [11]:
x_test2_gen = np.random.poisson(x_test)

for i in range(10):
    x_train2_gen = np.random.poisson(x_train2)
    
    model.fit(x_train2_gen, y_train2,
              validation_split=0.1,
               batch_size=64,
               epochs=1,
               verbose=1,
               shuffle = True
             )
model.evaluate(x_test2_gen, y_test)

3125/3125 [==============================] - 46s 15ms/step - loss: 1154.4673 - mean_squared_error: 1154.4673


[1154.46728515625, 1154.46728515625]

In [12]:
furthur_index = index
index = 1
path = '/home/ML4NO/ML/Regression/models_PoissonTrain/0910_theta23_{}_{}_{}.h5'
while os.path.isfile(path.format(model_index, furthur_index, index)):
    index += 1
model.save(path.format(model_index, furthur_index, index))